In [1]:
import pandas as pd
import numpy as np
from datetime import datetime

In [2]:
df = pd.read_csv('project_data/remuneracao_quadros_tecnicos.csv',
                sep=';', encoding='utf-8')

In [3]:
if 'Unnamed: 0' in df:
    df.drop('Unnamed: 0', axis = 1, inplace=True)

In [4]:
df.reset_index(drop=True,inplace=True)

In [5]:
df.dtypes

registro_funcional                   int64
num_vinculo                          int64
nome_servidor                       object
dc_cargo_base                       object
grupo_cargo_base                    object
nivel_cargo_base                    object
referencia_cargo_base               object
segmento_cargo_base                 object
dc_cargo_comissao                   object
referencia_cargo_comissao           object
escol_cargo_comissao                object
relacao_juridica                    object
qt_hora_jornada_semanal             object
dc_orgao                            object
dc_setor                            object
dc_orgao_externo                    object
qtd_servidores_orgao               float64
dc_tipo_logradouro_orgao            object
nm_logradouro_orgao                 object
nr_logradouro_orgao                float64
tx_complemento_logradouro_orgao     object
dt_inicio_exercicio                 object
ano_nascimento                       int64
sexo       

Arrumando valores de remuneracao para float

In [6]:
cols_remuneracao = ['vl_remuneracao_base_mensal',
                   'vl_outro_elemento_remuneracao',
                    'vl_remuneracao_total_bruta']

In [7]:
def remuneracao_float(item):
    
    if pd.isnull(item):
        return np.NaN
    
    item = item.replace('R$', '')
    item = item.replace('.', '')
    item = item.replace(',', '.')
    item = item.strip()
    
    return round(float(item),2)

In [8]:
for col in cols_remuneracao:
    df[col] = df[col].apply(remuneracao_float)

Avaliar relação entre descrições dos cargos. Acho que há sobreposição entre as informações das colunas

In [9]:
cols_desc_cargo_base = [
    'dc_cargo_base',
    'grupo_cargo_base', 
    'nivel_cargo_base', 
    'referencia_cargo_base',
    'segmento_cargo_base'
]

In [10]:
df[cols_desc_cargo_base].sample(5)

,dc_cargo_base,grupo_cargo_base,nivel_cargo_base,referencia_cargo_base,segmento_cargo_base
2029,"PROFISSIONAL ENG, ARQ, AGRONOMIA,GEOLOGIA",QEAG,II,QEAG6,AGRONOMIA
192,ANALISTA PLANEJAMENTO DESENV ORGANIZACIONAL,QAA,IV,Q16,CIENCIAS CONTABEIS
2208,ANALISTA ASSISTENCIA DESENVOLVIMENTO SOCIAL,QAA,I,Q5,SERVICO SOCIAL
2130,ANALISTA ASSISTENCIA DESENVOLVIMENTO SOCIAL,QAA,II,Q6,SERVICO SOCIAL
24,"PROFISSIONAL ENG, ARQ, AGRONOMIA,GEOLOGIA",QEAG,IV,QEAG17,ARQUITETURA


In [11]:
def remover_nivel_cargo_base(item):
    
    item = item.replace('-', '')
    
    return ''.join(char for char in item if not char.isdigit())

In [12]:
df['ref_cargo_base_s_nivel'] = df['referencia_cargo_base'].apply(remover_nivel_cargo_base)

In [13]:
pd.crosstab(df['grupo_cargo_base'],df['ref_cargo_base_s_nivel']).T

grupo_cargo_base,PCCS,QAA,QEAG,QPA,QPAT,QPCEL,QPDU,QPGG
ref_cargo_base_s_nivel,,,,,,,,
AMCI,0,0,0,0,0,0,0,70
APPGG,0,0,0,0,0,0,0,64
Q,0,1786,0,0,0,0,0,0
QAA,0,47,0,0,0,0,0,0
QCED,0,0,0,0,0,1,0,0
QEAG,0,0,927,0,0,0,0,0
QPAD,0,0,0,1,0,0,0,0
QPAE,0,0,0,1,0,0,0,0
QPAT,0,0,0,0,488,0,0,0


In [14]:
pd.crosstab(df['dc_cargo_base'],df['ref_cargo_base_s_nivel'])

ref_cargo_base_s_nivel,AMCI,APPGG,Q,QAA,QCED,QEAG,QPAD,QPAE,QPAT,QPDE,S
dc_cargo_base,,,,,,,,,,,
ANALISTA,0,0,0,32,0,0,0,0,0,0,0
ANALISTA ASSIST DESENV SOCIAL EQUIP SOCIAL,0,0,14,0,0,0,0,0,0,0,0
ANALISTA ASSISTENCIA DESENVOLVIMENTO SOCIAL,0,0,793,2,0,0,0,0,0,0,0
ANALISTA DE INFORMACOES CULTURA E DESPORTO,0,0,655,11,0,0,0,0,0,0,0
ANALISTA DE MEIO AMBIENTE,0,0,82,0,0,0,0,0,0,0,0
ANALISTA DE ORDENAMENTO TERRITORIAL,0,0,40,2,0,0,0,0,0,0,0
ANALISTA PLANEJAMENTO DESENV ORGANIZACIONAL,0,0,202,0,0,0,0,0,0,0,0
ANALISTA POLITICAS PUBLICAS GESTAO GOVERNAMENTAL,0,64,0,0,0,0,0,0,0,0,0
ARQUITETO,0,0,0,0,0,0,0,0,0,2,0


Há bastante sobreposição. Vamos manter apenas a descrição do cargo e o grupo.

In [15]:
df.drop(['ref_cargo_base_s_nivel', 'referencia_cargo_base'], axis =1, inplace=True)

Parece haver uma sobreposicao parecida no que diz respeito aos cargos em comissao.

Além disso, nosso conhecimento da área de negócio indica que, para os cargos em comissão, é preferível utilizar a referência.

In [16]:
cols_cargo_comissao = ['dc_cargo_comissao', 'referencia_cargo_comissao','escol_cargo_comissao']
df[df['dc_cargo_comissao'].notnull()][cols_cargo_comissao].sample(10)

,dc_cargo_comissao,referencia_cargo_comissao,escol_cargo_comissao
206,DIRETOR DE DIVISAO TECNICA,DAS12,SUPERIOR COMPLETO
1122,COORDENADOR V,DAS15,SUPERIOR COMPLETO
795,COORDENADOR,DAS10,SUPERIOR COMPLETO
518,DIRETOR DE DIVISAO TECNICA,DAS12,SUPERIOR COMPLETO
3076,ASSESSOR ESPECIAL II,DAS15,NAO SE APLICA
3223,SUPERVISOR TECNICO II,DAS12,SUPERIOR COMPLETO
2780,CHEFE DE UNIDADE TECNICA I,DAS10,NAO SE APLICA
2757,DIRETOR DE DIVISAO TECNICA,DAS12,SUPERIOR COMPLETO
2117,COORDENADOR I,DAS11,SUPERIOR COMPLETO
1736,ASSESSOR II,DAS10,SUPERIOR COMPLETO


In [17]:
cross_cargo_comissao = pd.crosstab(df['referencia_cargo_comissao'], df['dc_cargo_comissao'])

In [18]:
for col in cross_cargo_comissao:
    cross_cargo_comissao[col] = cross_cargo_comissao[col].apply(lambda x: True if x else False)


In [19]:
cross_cargo_comissao

dc_cargo_comissao,ADMINISTRADOR DE PARQUE II,ASSESSOR ESPECIAL I,ASSESSOR ESPECIAL II,ASSESSOR I,ASSESSOR II,ASSESSOR TECNICO I,ASSESSOR TECNICO II,CHEFE DE ASSESSORIA TECNICA II,CHEFE DE ASSESSORIA TECNICA III,CHEFE DE REPRESENTACAO FISCAL,...,COORDENADOR V,DIRETOR DE DEPARTAMENTO TECNICO,DIRETOR DE DIVISAO TECNICA,ENCARREGADO DE EQUIPE I,PRESIDENTE DE CONSELHO,REPRESENTANTE FISCAL,SECRETARIO EXECUTIVO ADJUNTO,SUBPREFEITO,SUBSECRETARIO,SUPERVISOR TECNICO II
referencia_cargo_comissao,,,,,,,,,,,,,,,,,,,,,
DAI06,False,False,False,False,False,False,False,False,False,False,...,False,False,False,True,False,False,False,False,False,False
DAS09,True,False,False,True,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
DAS10,False,False,False,False,True,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
DAS11,False,False,False,False,False,True,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
DAS12,False,False,False,False,False,False,True,False,False,False,...,False,False,True,False,False,True,False,False,False,True
DAS13,False,False,False,False,False,False,False,False,False,True,...,False,False,False,False,False,False,False,False,False,False
DAS14,False,True,False,False,False,False,False,True,False,False,...,False,True,False,False,False,False,False,False,False,False
DAS15,False,False,True,False,False,False,False,False,True,False,...,True,False,False,False,True,False,False,False,True,False
SAD,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,True,False,False,False


In [20]:
(cross_cargo_comissao.sum()==1).all()

True

Como não há nenhum cargo na base que corresponda a mais de uma referência, escohemos utilizar a referência do cargo, pois ela sintetiza melhor a relação entre responsabilidade e salário do que o nome do cargo.

In [21]:
df.drop('dc_cargo_comissao', axis=1,inplace=True)

Quando à escolaridade, alguns cargos em comissão podem prescindir da necessidade de ensino superior.
No entanto, isso não afeta os vencimentos. Por isso, vamos remover essa coluna também (sobretudo porque todos os servidores de nossa base possuem ensino superior completo).

In [22]:
pd.crosstab(df['referencia_cargo_comissao'], df['escol_cargo_comissao'])

escol_cargo_comissao,ENSINO MEDIO - COMPLETO,NAO SE APLICA,SUPERIOR COMPLETO
referencia_cargo_comissao,,,
ATC-1,0,0,65
ATC-2,0,0,32
ATC-3,0,0,2
ATC-4,0,0,5
DAI06,0,1,0
DAS09,0,12,6
DAS10,0,16,232
DAS11,2,3,145
DAS12,0,3,191


In [23]:
df.drop('escol_cargo_comissao', axis=1,inplace=True)

In [24]:
df['qt_hora_jornada_semanal'].unique()

array(['40 H', '30 H ESP ASST SOC', '44 H', '20 H', '30 H'], dtype=object)

In [25]:
def jornada_int(item):
    
    item = ''.join(char for char in item if char.isdigit())
    
    return int(item)

In [26]:
df['qt_hora_jornada_semanal'] = df['qt_hora_jornada_semanal'].apply(jornada_int)

Agora vamos dropar colunas que acreditamos que não tem relação com o modelo que queremos construir, como o endereço de trabalho, o nome do servidor  etc.. Vou deixar o Regsitro Funcional para poder tirar dúvidas no portal da transparência durante a exploratória

In [27]:
sem_relacao = [
    'nome_servidor',
    'dc_tipo_logradouro_orgao',
    'nr_logradouro_orgao',
    'nm_logradouro_orgao',
    'tx_complemento_logradouro_orgao',
    'dc_setor' #possui muitos termos, melhro ficar só com secretaria
    
]

In [28]:
df.drop(sem_relacao, axis=1, inplace=True)

Agora vamos criar uma variável "cedido", a partir da presença de um órgão externo.

In [29]:
df[df['dc_orgao_externo'].notnull()][['dc_orgao_externo', 'dc_orgao']]

,dc_orgao_externo,dc_orgao
0,TRIBUNAL DE CONTAS DO MUNICIPIO DE SAO PAULO -...,PROCURADORIA GERAL DO MUNICIPIO
7,CAMARA MUNICIPAL DE SAO PAULO,SECRETARIA MUNICIPAL DO VERDE E DO MEIO AMBIENTE
18,CAMARA MUNICIPAL DE SAO PAULO,GABINETE DO PREFEITO
50,CAMARA MUNICIPAL DE SAO PAULO,SECRETARIA MUNICIPAL DO VERDE E DO MEIO AMBIENTE
51,CAMARA MUNICIPAL DE SAO PAULO,SECRETARIA MUNICIPAL DE HABITACAO
...,...,...
2664,AUTORIDADE MUNICIPAL DE LIMPEZA URBANA,SECRETARIA MUNICIPAL DAS SUBPREFEITURAS
2982,HOSPITAL DAS CLINICAS,SECRETARIA MUNICIPAL DA SAUDE
3045,TRIBUNAL DE CONTAS DO MUNICIPIO DE SAO PAULO -...,GABINETE DO PREFEITO
3069,FUNDACAO THEATRO MUNICIPAL DE SAO PAULO,GABINETE DO PREFEITO


In [30]:
df['cedido']=df['dc_orgao_externo'].notnull()

In [31]:
df.drop('dc_orgao_externo', axis=1, inplace=True)

Vamos criar uma coluna de dias de exercício. E deletar a coluna categórica de data de início de exercício.

In [32]:
df['dt_inicio_exercicio'] = pd.to_datetime(df['dt_inicio_exercicio'], format= '%d/%m/%Y')

In [33]:
df['tempo_exercicio_dias'] = (datetime.today() - df['dt_inicio_exercicio']).apply(lambda x: x.days)

In [34]:
df[['dt_inicio_exercicio', 'tempo_exercicio_dias']].sample(5)

,dt_inicio_exercicio,tempo_exercicio_dias
2426,2012-08-16,3290
1609,2010-06-21,4077
2057,2014-09-09,2536
2313,2012-05-15,3383
2903,2015-09-18,2162


In [35]:
df.drop('dt_inicio_exercicio', axis=1, inplace=True)

In [36]:
df.to_csv('project_data/remuneracao_quadros_tecnicos_limpo.csv')